In [10]:
import os
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer, util
from typing import List
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
import tempfile

#### Understanding embeddings

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

query = "forgot my password"

docs = [
    "Password recovery: Use the 'Reset Password' link on login page",
    "Vacation policy: Request time off 2 weeks in advance",
    "Account security: Enable two-factor authentication",
    "Login help: contact IT if you cannot access your account" 
]

# Convert query and docs into embeddings
query_emb = model.encode(query)
doc_emb = model.encode(docs)

scores = util.cos_sim(query_emb,doc_emb)[0]

print(f"Query: {query}\n")
print("Results (score > 0.3 = relevant): ")

for doc, score in zip(docs, scores):
    marker = "✅" if score > 0.3 else " "
    print(f"{marker} [{score:.2f}] {doc}")

print("\n💡 Notice: Found 'Password recovery' and 'Login help'")
print(" Even though query didn't contain those exact words")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Query: forgot my password

Results (score > 0.3 = relevant): 
✅ [0.56] Password recovery: Use the 'Reset Password' link on login page
  [0.07] Vacation policy: Request time off 2 weeks in advance
✅ [0.31] Account security: Enable two-factor authentication
✅ [0.60] Login help: contact IT if you cannot access your account

💡 Notice: Found 'Password recovery' and 'Login help'
 Even though query didn't contain those exact words


#### 2. Document Processing with Smart Chunking

In [7]:
from torch import long


print("="*55)

long_document = """
TechDocs Employee Handbook

    Chapter 1: Remote Work Policy

    Our company embraces flexible work arrangements to support work-life balance.
    Employees are permitted to work remotely up to 3 days per week, provided they
    maintain regular communication with their team and meet all performance expectations.
    Remote work requires manager approval and must not impact team collaboration or
    customer service quality.

    To work remotely, employees must have a suitable home office setup with reliable
    internet connection, appropriate workspace, and necessary equipment. The company
    provides a one-time stipend of $500 for home office setup. VPN access is mandatory
    for accessing company systems remotely.

    Chapter 2: Communication Guidelines

    Effective communication is essential for remote work success. All employees must
    be available during core hours (10 AM - 3 PM in their local timezone) for meetings
    and collaboration. Slack is our primary communication tool, with response times
    expected within 2 hours during work hours.

    Video calls are encouraged for team meetings to maintain personal connections.
    Camera use is optional but recommended. All meetings should have clear agendas
    and action items documented in our project management system.

    Chapter 3: Performance Management

    Remote work does not change performance expectations. Managers will evaluate
    employees based on deliverables, quality of work, and contribution to team goals
    rather than hours logged. Regular 1-on-1 meetings should continue virtually.
"""

print(f"Original document length: {len(long_document)} characters")

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    length_function=len,
    separators=[" "]
)

# Split the document
chunks = splitter.split_text(long_document)

print(f"\nChunking results:")
print(f". Created {len(chunks)} chunks")
print(f". Chunk size: ~500 characters")
print(f". Overlap: 100 characters\n")

for i, chunk in enumerate(chunks[:3],1):
    print(f"Chunk {i} ({len(chunk)} chars):")
    print("-"*40)
    print(chunk[:200] + "..." if len(chunk) > 200 else chunk)


# Analyze overlaps
if len(chunk) > 1:
    overlap_start = chunks[1][:100]
    print("-"*40)
    print(f"Found overlap: {overlap_start[:50]}...")
    print("Context preservered between chunks")

Original document length: 1599 characters

Chunking results:
. Created 5 chunks
. Chunk size: ~500 characters
. Overlap: 100 characters

Chunk 1 (496 chars):
----------------------------------------
TechDocs Employee Handbook

    Chapter 1: Remote Work Policy

    Our company embraces flexible work arrangements to support work-life balance.
    Employees are permitted to work remotely up to 3 da...
Chunk 2 (499 chars):
----------------------------------------
expectations.
    Remote work requires manager approval and must not impact team collaboration or
    customer service quality.

    To work remotely, employees must have a suitable home office setup ...
Chunk 3 (492 chars):
----------------------------------------
provides a one-time stipend of $500 for home office setup. VPN access is mandatory
    for accessing company systems remotely.

    Chapter 2: Communication Guidelines

    Effective communication is ...
----------------------------------------
Found overlap: expectati

#### 3. Building Vectorstore with ChromaDB

In [12]:
from sympy import per


embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print("Embedding Model loaded")

document_text = ["""Remote Work Policy: Employees can work from home up to 3 days per week.
        VPN access required. Core hours 10 AM - 3 PM local time. Manager approval needed.""",

        """Office Benefits: Free lunch on Tuesdays and Thursdays. Gym membership reimbursement
        up to $50/month. Annual learning budget of $2000 for courses and conferences.""",

        """Vacation Policy: 15 days PTO for first year, 20 days after 2 years.
        Sick leave separate - 10 days per year. Holidays follow local calendar.""",

        """Security Guidelines: Two-factor authentication required for all accounts.Password changes every 90 days. No sharing of credentials. Report incidents immediately""",
        
        """Dress Code: Business casual Monday-Thursday. Casual Fridays allow jeans. Client meetings require business formal. Work from home has no dress code requirements."""]


for i, text in enumerate(document_text):
    doc = Document(
        page_content=text,
        metadata={
            "source": f"handbook_section_{i+1}",
            "type": "policy",
            "id": 1,   
        }
    )
    document.append(doc)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)
splits = text_splitter.split_documents(document)
print(f"Created {len(splits)} chunks")

with tempfile.TemporaryDirectory() as temp_dir:
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_directory=temp_dir,
        collection_name="techdocs"
    )
    print(f"Vector store created with {vectorstore._collection.count()} vectors")

    test_queries = [
        "Can I work from home?",
        "What's the dress code for Friday?",
        "How many vaction days do I get?"
    ]

    for query in test_queries:
        print(f"\n Query: {query}")
        results = vectorstore.similarity_search(query, k=1)
        if results:
            print(f"Best match: {results[0].page_content[:100]}...")
            print(f"    Source: {results[0].metadata.get('source','unknown')}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embedding Model loaded
Created 5 chunks
Vector store created with 5 vectors

 Query: Can I work from home?
Best match: Remote Work Policy: Employees can work from home up to 3 days per week.
        VPN access required....
    Source: handbook_section_1

 Query: What's the dress code for Friday?
Best match: Dress Code: Business casual Monday-Thursday. Casual Fridays allow jeans. Client meetings require bus...
    Source: handbook_section_5

 Query: How many vaction days do I get?
Best match: Vacation Policy: 15 days PTO for first year, 20 days after 2 years.
        Sick leave separate - 10...
    Source: handbook_section_3


#### 4. Semantic search

In [16]:
from sklearn.linear_model import enet_path


knowledge_base = [
        "Remote work policy allows employees to work from home up to 3 days per week with manager approval.",
        "Dress code is business casual Monday-Thursday. Jeans are permitted on Fridays only.",
        "Annual performance reviews happen in December with mid-year check-ins in June.",
        "Health insurance covers employee and dependents with company paying 80% of premiums.",
        "401k retirement plan includes company match up to 6% of salary.",
        "Vacation policy provides 15 days PTO first year, increasing to 20 days after 2 years.",
        "Sick leave is separate from vacation with 10 days allocated annually.",
        "Training budget of $2000 per employee for professional development courses.",
        "Office provides free lunch on Tuesdays and Thursdays for all employees.",
        "Parking is free for all employees in the company garage.",
        "Work hours are flexible but core hours 10 AM to 3 PM are required.",
        "Security policy requires password changes every 90 days and two-factor authentication."
    ]

documents = [Document(page_content=text, metadata={"id":1})
             for i, text in enumerate(knowledge_base)]

print(f"Loading {len(documents)} documents into vector store...")

# Create vector store
with tempfile.TemporaryDirectory() as temp_dir:
    vectorstore=Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory=temp_dir
    )
    
    search_query = "work from home policy"
    
    # Number of results
    k=3

    print(f"🔎Searching for: '{search_query}'")
    print(f"    Returning top {k} results")
    print("-"*40)

    results=vectorstore.similarity_search(search_query,k=k)

    print("\nSearch Results:")
    for i, doc in enumerate(results,1):
        print(f"\n{i}.{doc.page_content}")

    score_threshold = 1.3
    print(f"Filtered search (search(threshold > {score_threshold}):")

    results_with_score=vectorstore.similarity_search_with_score(
        search_query,
        k=5
    )   

    relevant_results = [(doc, score) for doc, score in results_with_score 
                        if score <= score_threshold]

    if relevant_results:
        for doc, score in relevant_results[:3]:
            print(f"\nScore: {score:.3f}")
            print(f"    {doc.page_content[:100]}...")

    else:
        print("No results above threshold")
    

Loading 12 documents into vector store...
🔎Searching for: 'work from home policy'
    Returning top 3 results
----------------------------------------

Search Results:

1.Remote work policy allows employees to work from home up to 3 days per week with manager approval.

2.Work hours are flexible but core hours 10 AM to 3 PM are required.

3.Health insurance covers employee and dependents with company paying 80% of premiums.
Filtered search (search(threshold > 1.3):

Score: 0.707
    Remote work policy allows employees to work from home up to 3 days per week with manager approval....


In [18]:
# Advanced search demonstrations
test_searches = [
    ("Can I bring my dog to work?", "dress code"),
    ("How many days off?", "vacation"),
    ("retirement savings", "401k")
]

for query, expected_topic in test_searches:
    results_1=vectorstore.similarity_search(query,k=1)
    found_topic=expected_topic.lower() in results_1[0].page_content.lower()
    status="✅" if found_topic else "❌"
    print(f"{status} '{query}' -> Found: {expected_topic}")


❌ 'Can I bring my dog to work?' -> Found: dress code
✅ 'How many days off?' -> Found: vacation
✅ 'retirement savings' -> Found: 401k
